In [38]:
import pandas as pd

In [39]:
file_all_tes = '../../data/TEs/all.gff3'
file_all_mite = '../../data/TEs/all_filter_mite.gff3'
file_ann = '../../data/genome/annotations/introns.gff3'

In [40]:
df = pd.read_csv(file_all_tes, index_col=False, sep='\t', comment='#', header=None)
df.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']


/home/juan/Desktop/juan/bio/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
print(len(df.index))

1745371


In [42]:
df.head(4)

,seqname,source,feature,start,end,score,strand,frame,attribute
0,1A,TE,TE,77566,78177,.,+,.,MITE_T_122702|chr2B|476704370|476705072|AT|10|...
1,1A,TE,TE,86285,86766,.,+,.,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...
2,1A,TE,TE,213867,214194,.,+,.,MITE_T_8262|chr5A|273309274|273309638|TA|373|F468
3,1A,TE,TE,219804,220469,.,+,.,MITE_T_82582|chr3D|312088827|312089549|AT|18|F...


In [43]:
df = df[(df.attribute.str.startswith('MITE')) | (df.attribute.str.startswith('DT'))]

In [44]:
print(len(df.index))

1222547


In [60]:
df.head(4)

,index,seqname,source,feature,start,end,score,strand,frame,attribute
0,0,1A,TE,exon,77566,78177,.,+,.,MITE_T_122702|chr2B|476704370|476705072|AT|10|...
1,1,1A,TE,intron,86285,86766,.,+,.,MITE_T_126801|chr7B|78567635|78568204|CTTCCCT|...
2,2,1A,TE,,213867,214194,.,+,.,MITE_T_8262|chr5A|273309274|273309638|TA|373|F468
3,3,1A,TE,,219804,220469,.,+,.,MITE_T_82582|chr3D|312088827|312089549|AT|18|F...


In [57]:
df.reset_index(inplace=True)

In [58]:
df.to_csv(file_all_mite, sep='\t', index=None, header=None)

In [51]:
df_ann = pd.read_csv(file_ann, index_col=False, sep='\t', comment='#', header=None)
df_ann.columns = ['seqname' , 'source' , 'feature' , 'start' , 'end' , 'score' , 'strand' , 'frame' , 'attribute']


In [52]:
print(len(df_ann.index))

2573233


In [53]:
df_ann = df_ann[
    (df_ann.feature == 'intron') | 
    (df_ann.feature == 'exon') | 
    (df_ann.feature == 'three_prime_UTR') | 
    (df_ann.feature == 'five_prime_UTR')]

In [54]:
print(len(df_ann.index))

1593666


In [ ]:
howfar = 200
df['feature'] = ''
rows = []
for k_mite, v_mite in df.iterrows():
    if k_mite % 100 == 0:
        print(k_mite)
    for k_feature,v_feature in df_ann[df_ann.seqname == v_mite.seqname].iterrows():
        start = max(int(v_feature.start) - howfar, 0)
        end = int(v_feature.end) + howfar
        #overlaps with gene
        if int(v_mite.end) >= start and v_mite.start <= end:
            df.at[k_mite, 'feature'] = v_feature.feature

0
